# CycleGAN 

In [1]:
from experiment_utils import ExperimentRun
import tensorflow as tf
import matplotlib.pyplot as plt
from models import build_generator, build_discriminator
from dataset import get_dataset
import os
import time, sys

# os.makedirs("results", exist_ok=True)

2025-11-19 02:20:18.031326: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-19 02:20:18.096240: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-19 02:20:19.683780: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Hyperparameters

In [2]:
lambda_cyc, lambda_id = 5.0, 2.5
img_size = 256
batch_size = 2
num_epochs = 100
trainX_pattern = '../../data/animals_balanced/butterfly/*.JPEG'
trainY_pattern = '../../data/origami_images/butterfly/*.jpg'

### Generator & Discriminator

In [3]:
G = build_generator(image_size=img_size, n_blocks=9)
F = build_generator(image_size=img_size, n_blocks=9)
D_X = build_discriminator(image_size=img_size)
D_Y = build_discriminator(image_size=img_size)

I0000 00:00:1763518825.935472    2613 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5



### Optimizer & Loss

In [4]:
G_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
D_X_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
D_Y_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

mse = tf.keras.losses.MeanSquaredError()
L1 = lambda a, b: tf.reduce_mean(tf.abs(a - b))

### Setup for Experiments

In [5]:
#expereiment setup
params = {
    "lambda_cyc": lambda_cyc,
    "lambda_id": lambda_id,
    "img_size": img_size,
    "batch_size": batch_size,
    "num_epochs": num_epochs,
    "G_lr": 1e-4,
    "D_lr": 1e-4,
    "n_blocks": 9,
}

config = {
    **params,
    "trainX_pattern": trainX_pattern,
    "trainY_pattern": trainY_pattern,
}

exp = ExperimentRun(params=params, config=config)
logger = exp.get_logger()

# connect checkpoint manager
ckpt, ckpt_manager = exp.create_checkpoint_manager(
    G=G, F=F, D_X=D_X, D_Y=D_Y,
    G_optimizer=G_optimizer,
    D_X_optimizer=D_X_optimizer,
    D_Y_optimizer=D_Y_optimizer,
)

2025-11-19 02:20:30,156 - INFO - Experiment directory: experiments/run_20251119-022030


### Training Step

In [6]:
# ----------------------------
# 3️⃣ 1 training step
# ----------------------------
@tf.function
def train_step(real_x, real_y, lambda_cyc, lambda_id):
    with tf.GradientTape(persistent=True) as tape:

        fake_y = G(real_x, training=True)
        fake_x = F(real_y, training=True)

        cyc_x = F(fake_y, training=True)
        cyc_y = G(fake_x, training=True)

        same_x = F(real_x, training=True)
        same_y = G(real_y, training=True)

        D_X_real = D_X(real_x, training=True)
        D_X_fake = D_X(fake_x, training=True)
        D_Y_real = D_Y(real_y, training=True)
        D_Y_fake = D_Y(fake_y, training=True)

        G_GAN_loss = mse(tf.ones_like(D_Y_fake), D_Y_fake)
        F_GAN_loss = mse(tf.ones_like(D_X_fake), D_X_fake)

        D_X_loss = 0.5 * (mse(tf.ones_like(D_X_real), D_X_real) + mse(tf.zeros_like(D_X_fake), D_X_fake))
        D_Y_loss = 0.5 * (mse(tf.ones_like(D_Y_real), D_Y_real) + mse(tf.zeros_like(D_Y_fake), D_Y_fake))

        cycle_loss = L1(cyc_x, real_x) + L1(cyc_y, real_y)
        id_loss = L1(same_x, real_x) + L1(same_y, real_y)

        G_total_loss = G_GAN_loss + F_GAN_loss + lambda_cyc * cycle_loss + lambda_id * id_loss

    G_grads = tape.gradient(G_total_loss, G.trainable_variables + F.trainable_variables)
    D_X_grads = tape.gradient(D_X_loss, D_X.trainable_variables)
    D_Y_grads = tape.gradient(D_Y_loss, D_Y.trainable_variables)

    del tape
    G_optimizer.apply_gradients(zip(G_grads, G.trainable_variables + F.trainable_variables))
    D_X_optimizer.apply_gradients(zip(D_X_grads, D_X.trainable_variables))
    D_Y_optimizer.apply_gradients(zip(D_Y_grads, D_Y.trainable_variables))

    return G_total_loss, D_X_loss, D_Y_loss


### Load Dataset

In [7]:
trainX = get_dataset(trainX_pattern, batch_size=batch_size, img_size=img_size, augment=True)
trainY = get_dataset(trainY_pattern, batch_size=batch_size, img_size=img_size, augment=True)

### Training Cycle

In [8]:
cardX = tf.data.experimental.cardinality(trainX).numpy()
cardY = tf.data.experimental.cardinality(trainY).numpy()
num_steps_per_epoch = int(min(cardX, cardY))
bar_len = 30
ckpt_interval = 5

for epoch in range(num_epochs):
    
    epoch_start = time.time() 
    
    logger.info(f"Epoch {epoch+1}/{num_epochs}")
    step = 0
    
    for real_x, real_y in zip(trainX, trainY):
        step += 1
        G_loss, DX_loss, DY_loss = train_step(real_x, real_y, lambda_cyc, lambda_id)
        
        bar = exp.progress_bar(step, num_steps_per_epoch, bar_len=bar_len)
        sys.stdout.write(
            "\r"
            f"Epoch {epoch+1}/{num_epochs} "
            f"{bar}  step {step}/{num_steps_per_epoch}"
        )
        sys.stdout.flush()
        
        # if step % 50 == 0:
        #     logger.info(
        #         f"  step {step}: G={float(G_loss.numpy()):.3f} "
        #         f"DX={float(DX_loss.numpy()):.3f} DY={float(DY_loss.numpy()):.3f}"
        #     )
        loss_dict = {
            "G_total": G_loss,
            "D_X": DX_loss,
            "D_Y": DY_loss,
        }
        exp.log_losses(epoch + 1, step, loss_dict)
        
    sys.stdout.write("\n")
    sys.stdout.flush()
    
    epoch_time = time.time() - epoch_start
    time_per_step = epoch_time / step

    logger.info(
        f"Epoch {epoch+1} completed in {epoch_time:.2f}s "
        f"({time_per_step:.4f}s per step)"
    )
    
    if (epoch + 1) % ckpt_interval == 0 or (epoch + 1) == num_epochs:
        exp.save_checkpoint()

    sample = next(iter(trainX))
    fake_y = G(sample, training=False)

    if (epoch + 1) % 10 == 0:
        logger.info(
            "Epoch %d: G=%.3f DX=%.3f DY=%.3f",
            epoch + 1, float(G_loss.numpy()), float(DX_loss.numpy()), float(DY_loss.numpy())
        )
        sample = next(iter(trainX))
        fake_y = G(sample, training=False)
        plt.figure(figsize=(6, 3))
        plt.subplot(1, 2, 1); plt.imshow((sample[0] + 1) / 2); plt.title("Real X"); plt.axis("off")
        plt.subplot(1, 2, 2); plt.imshow((fake_y[0] + 1) / 2); plt.title("Fake Y"); plt.axis("off")
        plt.tight_layout()
        out_path = exp.results_dir / f"epoch_{epoch+1}.png"
        plt.savefig(out_path)
        plt.close()
        
print("\nGenerating final comparison on test image...")



2025-11-19 02:20:34,931 - INFO - Epoch 1/100
2025-11-19 02:21:19.502135: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 90501


Epoch 1/100 [==============================] 100.0%  step 52/52

2025-11-19 02:22:55.403204: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


2025-11-19 02:22:56,955 - INFO - Epoch 1 completed in 142.02s (2.7312s per step)
2025-11-19 02:22:58,930 - INFO - Epoch 2/100


Epoch 2/100 [==============================] 100.0%  step 52/52

...

Epoch 99/100 [==============================] 100.0%  step 52/52


2025-11-19 04:15:57,238 - INFO - Epoch 99 completed in 67.49s (1.2980s per step)
INFO:run_20251119-022030:Epoch 99 completed in 67.49s (1.2980s per step)
2025-11-19 04:15:58,570 - INFO - Epoch 100/100
INFO:run_20251119-022030:Epoch 100/100


Epoch 100/100 [==============================] 100.0%  step 52/52


2025-11-19 04:17:06,059 - INFO - Epoch 100 completed in 67.49s (1.2979s per step)
INFO:run_20251119-022030:Epoch 100 completed in 67.49s (1.2979s per step)
2025-11-19 04:17:06,968 - INFO - Saved checkpoint: experiments/run_20251119-022030/checkpoints/ckpt-20
INFO:run_20251119-022030:Saved checkpoint: experiments/run_20251119-022030/checkpoints/ckpt-20
2025-11-19 04:17:08,123 - INFO - Epoch 100: G=3.629 DX=0.287 DY=0.102
INFO:run_20251119-022030:Epoch 100: G=3.629 DX=0.287 DY=0.102



Generating final comparison on test image...


### Test on 1 image

In [10]:
def load_test_image(path, img_size=256):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    img = (tf.cast(img, tf.float32) / 127.5) - 1.0   # scale [-1,1]
    return tf.expand_dims(img, axis=0)  # shape [1,256,256,3]

def save_output(fake, out_path):
    fake = (fake[0] + 1.0) / 2.0   # back to [0,1]
    plt.imsave(out_path, fake.numpy())

ckpt.restore(ckpt_manager.latest_checkpoint).expect_partial()
print("Loaded checkpoint:", ckpt_manager.latest_checkpoint)

test_image_path = "../NST/test_imgs/butterfly.jpg"
output_path = exp.results_dir / "output_test.png"

img = load_test_image(test_image_path, img_size=256)
fake = G(img, training=False)
save_output(fake, output_path)

print(f"Saved output to: {output_path}")

Loaded checkpoint: experiments/run_20251119-022030/checkpoints/ckpt-20
Saved output to: experiments/run_20251119-022030/results/output_test.png


In [14]:
experiment_root = exp.results_dir.parent
export_dir = experiment_root / "exported"
export_dir.mkdir(parents=True, exist_ok=True)

G.save(str(export_dir / "G_full.keras"))
F.save(str(export_dir / "F_full.keras"))

G.save_weights(str(export_dir / "G_full.weights.h5"))
F.save_weights(str(export_dir / "F_full.weights.h5"))

G.export(str(export_dir / "G_savedmodel"))
F.export(str(export_dir / "F_savedmodel"))

print("\nSaved model artifacts:")
print("  G_full.keras")
print("  F_full.keras")
print("  G_full.weights.h5")
print("  F_full.weights.h5")
print("  G_savedmodel/")
print("  F_savedmodel/")

INFO:tensorflow:Assets written to: experiments/run_20251119-022030/exported/G_savedmodel/assets


INFO:tensorflow:Assets written to: experiments/run_20251119-022030/exported/G_savedmodel/assets


Saved artifact at 'experiments/run_20251119-022030/exported/G_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None)
Captures:
  135427582572352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427582823232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427582825696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427582827456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427582830976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427582831680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427582832032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427582835024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427583167120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427583166064: TensorSpec(shape=(),

INFO:tensorflow:Assets written to: experiments/run_20251119-022030/exported/F_savedmodel/assets


Saved artifact at 'experiments/run_20251119-022030/exported/F_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name='keras_tensor_74')
Output Type:
  TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None)
Captures:
  135427499222016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427499431488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427499427792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427499430080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427499435712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427499436416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427499436768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427499439760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427499441696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135427372584624: TensorSpec(shape=